In [1]:
import requests
import json
import pandas as pd
import glob
import os
from scipy.stats import linregress
from sklearn import linear_model
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
import gmaps
import numpy
#from config import covid_api
#from config import google_api



# NOTES:
# https://api.covid19api.com/world/total
# This is all data - returns 10MB+
# https://api.covid19api.com/all


In [2]:
# From COVID-19 API
# https://covid19api.com/

#base_url = 'https://api.covid19api.com/summary'
#headers = {"X-Access-Token":covid_api}

#response = requests.get(base_url, headers=headers).json()
#print(json.dumps(response, indent=4, sort_keys=True))
#response['Countries']

In [3]:
# -----------------------
# Get risk factor data for North Carolina, based on census data
# THIS IS WORKING PROPERLY
# -----------------------

# Import CRE csv from
# https://www2.census.gov/data/experimental-data-products/community-resilience-estimates/2020/

#risk_path = "https://www2.census.gov/data/experimental-data-products/community-resilience-estimates/2020/cre-2018-a11.csv"
# If the above breaks, use the downloaded CSV
risk_path = "data/cre-2018-a11.csv"
risk_df = pd.read_csv(risk_path)

# Break down by North Carolina
nc_risk_df = risk_df.loc[risk_df['STABREV']=='NC']

# Display dataframe
nc_risk_df

,state,county,tract,rfgrp,prednum,prednum_moe,predrt,predrt_moe,stname,STABREV,ctname,popuni
150009,37,1,0,0RF,48284,9880.2,29.84,6.11,North Carolina,NC,"Alamance County, NC",161790
150010,37,1,0,1-2RF,74707,10852.5,46.18,6.71,North Carolina,NC,"Alamance County, NC",161790
150011,37,1,0,3PLRF,38799,9176.3,23.98,5.67,North Carolina,NC,"Alamance County, NC",161790
150012,37,1,20100,0RF,1122,385.8,26.26,9.03,North Carolina,NC,"Alamance County, NC",4272
150013,37,1,20100,1-2RF,2003,410.3,46.89,9.60,North Carolina,NC,"Alamance County, NC",4272
...,...,...,...,...,...,...,...,...,...,...,...,...
156889,37,199,960300,1-2RF,1289,301.9,42.19,9.88,North Carolina,NC,"Yancey County, NC",3055
156890,37,199,960300,3PLRF,827,283.9,27.07,9.29,North Carolina,NC,"Yancey County, NC",3055
156891,37,199,960400,0RF,1135,358.4,26.43,8.34,North Carolina,NC,"Yancey County, NC",4295
156892,37,199,960400,1-2RF,1998,401.6,46.52,9.35,North Carolina,NC,"Yancey County, NC",4295


In [4]:
# -----------------------
# Clean up the data
# THIS IS WORKING PROPERLY
# -----------------------
nc_risk_df = nc_risk_df[['tract','rfgrp','prednum','ctname','popuni']]
nc_risk_rename_df = nc_risk_df.rename(columns={
    'rfgrp':'risk_group',
    'prednum':'risk_pop',
    'ctname':'county',
    'popuni':'total_pop'})

nc_risk_rename_df['county'].replace(' County, NC','',regex=True,inplace=True)

# Now we only need to keep the data where tract = 0, as that's the county summary
nc_risk_grouped = nc_risk_rename_df.groupby(['county','tract','risk_group']).sum()
nc_risk_grouped.reset_index(inplace=True)
nc_risk_filtered = nc_risk_grouped[nc_risk_grouped['tract'] == 0] 

# Display the dataframe
nc_risk_filtered = nc_risk_filtered[['county','risk_group','risk_pop','total_pop']]
nc_risk_filtered

,county,risk_group,risk_pop,total_pop
0,Alamance,0RF,48284,161790
1,Alamance,1-2RF,74707,161790
2,Alamance,3PLRF,38799,161790
111,Alexander,0RF,9789,36096
112,Alexander,1-2RF,17442,36096
...,...,...,...,...
6844,Yadkin,1-2RF,16689,37498
6845,Yadkin,3PLRF,10706,37498
6867,Yancey,0RF,4307,17870
6868,Yancey,1-2RF,8271,17870


In [5]:
# -----------------------
# Reports statewide North Carolina risk factor groups
# THIS IS WORKING PROPERLY
# -----------------------

# Make a new dataframe from the other
nc_overall_risk_df = nc_risk_filtered

# Group by risk factors
nc_overall_risk = nc_overall_risk_df.groupby('risk_group').sum()

# Build new series to hold percentages
nc_overall_risk['percentage'] = ''

# Iterate over rows to calculate percentages, round, and store in the new series 
for index, row in nc_overall_risk.iterrows():
    nc_overall_risk.at[index,'percentage'] = round((row['risk_pop'] / row['total_pop'])*100,2)

# Display the dataframe
nc_overall_risk

,risk_pop,total_pop,percentage
risk_group,,,
0RF,2896542,10219101,28.34
1-2RF,4866961,10219101,47.63
3PLRF,2455598,10219101,24.03


In [6]:
# -----------------------
# Reports fisk factor groups, by county, for North Carolina, based on census data
# THIS IS WORKING PROPERLY
# -----------------------

nc_county_risk = nc_risk_filtered

# Build new series to hold percentages
nc_county_risk['percentage'] = ''

# Iterate over rows to calculate percentages, round, and store in the new series 
for index, row in nc_county_risk.iterrows():
    nc_county_risk.at[index,'percentage'] = round((row['risk_pop'] / row['total_pop'])*100,2)

# Display the dataframe
nc_county_risk


,county,risk_group,risk_pop,total_pop,percentage
0,Alamance,0RF,48284,161790,29.84
1,Alamance,1-2RF,74707,161790,46.18
2,Alamance,3PLRF,38799,161790,23.98
111,Alexander,0RF,9789,36096,27.12
112,Alexander,1-2RF,17442,36096,48.32
...,...,...,...,...,...
6844,Yadkin,1-2RF,16689,37498,44.51
6845,Yadkin,3PLRF,10706,37498,28.55
6867,Yancey,0RF,4307,17870,24.1
6868,Yancey,1-2RF,8271,17870,46.28


In [7]:
# -----------------------
# This is a NYTimes report on national COVID-19 cases and deaths, sorted by county
# From: https://github.com/nytimes/covid-19-data (us-counties.csv)
# THIS IS WORKING PROPERLY
# -----------------------

# Pull in the live data
counties_path = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
# If the above breaks, we can refer to the saved version of the csv, below:
#counties_path = "data/us-counties.csv"

# Save to a dataframe
covid_df = pd.read_csv(counties_path)

# Sort for NC
covid_county_df = covid_df.loc[covid_df['state']=='North Carolina']

# Drop irrelevant column(s)
covid_county_clean_df = covid_county_df.drop(['fips'], 1)

# Grab last 100 rows (for 100 counties to get latest data)
covid_county_latest_df = covid_county_clean_df.tail(100)

# Diplay the dataframe
covid_county_latest_df

,date,county,state,cases,deaths
355441,2020-07-21,Alamance,North Carolina,1755,40
355442,2020-07-21,Alexander,North Carolina,221,2
355443,2020-07-21,Alleghany,North Carolina,52,0
355444,2020-07-21,Anson,North Carolina,267,1
355445,2020-07-21,Ashe,North Carolina,77,1
...,...,...,...,...,...
355536,2020-07-21,Wayne,North Carolina,2123,35
355537,2020-07-21,Wilkes,North Carolina,665,7
355538,2020-07-21,Wilson,North Carolina,1174,28
355539,2020-07-21,Yadkin,North Carolina,415,6


In [8]:
# -----------------------
# Build a dataframe that shows worst 10 counties in cases and deaths
# THIS IS WORKING PROPERLY
# -----------------------

# Sort by cases, highest to lowest
sorted_cases = covid_county_latest_df.sort_values(by=['cases'],ascending=False).reset_index()
# Sort by deaths, highest to lowest
sorted_deaths = covid_county_latest_df.sort_values(by=['deaths'],ascending=False).reset_index()

# Build a new dataframe to hold top counties and the value, sorted by cases and deaths
top_df = pd.DataFrame(columns=['Most cases','Case count','Most deaths','Death count'])

# Add the cases to the new dataframe
top_df['Most cases'] = sorted_cases['county'].values
top_df['Case count'] = sorted_cases['cases'].values

# Add the deaths to the new dataframe
top_df['Most deaths'] = sorted_deaths['county'].values
top_df['Death count'] = sorted_deaths['deaths'].values

# Show the top 10 counties for cases and deaths
top_df.head(10)

,Most cases,Case count,Most deaths,Death count
0,Mecklenburg,17800,Mecklenburg,184
1,Wake,9143,Guilford,134
2,Durham,5219,Wake,99
3,Guilford,4314,Durham,73
4,Forsyth,4297,Henderson,52
5,Johnston,2437,Buncombe,51
6,Gaston,2359,Robeson,49
7,Union,2251,Chatham,47
8,Wayne,2123,Rowan,46
9,Cumberland,2032,Cumberland,46


In [9]:
# -----------------------
# Combine dataframes from NYTimes data and CRE data
# NYT = covid_county_latest_df
# CRE = nc_county_risk
# THIS IS WORKING PROPERLY
# -----------------------

# Do the merge
combined_data = pd.merge(nc_county_risk, covid_county_latest_df, on='county', how='outer')

# Display the dataframe
combined_data


,county,risk_group,risk_pop,total_pop,percentage,date,state,cases,deaths
0,Alamance,0RF,48284,161790,29.84,2020-07-21,North Carolina,1755,40
1,Alamance,1-2RF,74707,161790,46.18,2020-07-21,North Carolina,1755,40
2,Alamance,3PLRF,38799,161790,23.98,2020-07-21,North Carolina,1755,40
3,Alexander,0RF,9789,36096,27.12,2020-07-21,North Carolina,221,2
4,Alexander,1-2RF,17442,36096,48.32,2020-07-21,North Carolina,221,2
...,...,...,...,...,...,...,...,...,...
295,Yadkin,1-2RF,16689,37498,44.51,2020-07-21,North Carolina,415,6
296,Yadkin,3PLRF,10706,37498,28.55,2020-07-21,North Carolina,415,6
297,Yancey,0RF,4307,17870,24.1,2020-07-21,North Carolina,68,0
298,Yancey,1-2RF,8271,17870,46.28,2020-07-21,North Carolina,68,0


In [10]:
# -----------------------
# Build dataframe to show:
#     Cases and deaths as a percentage of the county population
#     Deaths as a percentage of cases
# THIS IS WORKING PROPERLY
# -----------------------

# Build empty dataframe and order columns
summary_df = pd.DataFrame(columns={
    'county',
    'cases_pop_pct',
    'deaths_cases_pct',
    'deaths_pop_pct',
    'cases',
    'deaths',
    'population'})
summary_df = summary_df[[
    'county','cases','deaths','population','cases_pop_pct','deaths_pop_pct','deaths_cases_pct']]


# Set empty lists to store values
county_lst = []
cases_lst = []
deaths_lst = []
pop_lst = []

# This variable checks to see if we've seen this county already
county = ''

#Iterate through the data and build lists to go into the new summary dataframe
for i, row in combined_data.iterrows():
    if county == row[0]: # This handles 2nd and 3rd occurrance of county
        pass
    else: # This handles first occurrance of county
        county_lst.append(row[0])
        cases_lst.append(row[7])
        deaths_lst.append(row[8])
        pop_lst.append(row[3])
        
        county = row[0]

# Build the summary dataframe
summary_df['county'] = county_lst
summary_df['cases'] = cases_lst
summary_df['deaths'] = deaths_lst
summary_df['population'] = pop_lst

# Do calculations for percentages
for i, row in summary_df.iterrows():
    summary_df.loc[i,'cases_pop_pct']= f'{round(((row[1]/row[3])*100),4)}%'
    summary_df.loc[i,'deaths_pop_pct']= f'{round(((row[2]/row[3])*100),4)}%'
    summary_df.loc[i,'deaths_cases_pct']= f'{round(((row[2]/row[1])*100),4)}%'

# Display the summary dataframe   
summary_df

,county,cases,deaths,population,cases_pop_pct,deaths_pop_pct,deaths_cases_pct
0,Alamance,1755,40,161790,1.0847%,0.0247%,2.2792%
1,Alexander,221,2,36096,0.6123%,0.0055%,0.905%
2,Alleghany,52,0,11146,0.4665%,0.0%,0.0%
3,Anson,267,1,22949,1.1634%,0.0044%,0.3745%
4,Ashe,77,1,27068,0.2845%,0.0037%,1.2987%
...,...,...,...,...,...,...,...
95,Wayne,2123,35,122647,1.731%,0.0285%,1.6486%
96,Wilkes,665,7,68118,0.9762%,0.0103%,1.0526%
97,Wilson,1174,28,80922,1.4508%,0.0346%,2.385%
98,Yadkin,415,6,37498,1.1067%,0.016%,1.4458%


In [11]:
# -----------------------
# COVID testing location data
# From https://github.com/codersagainstcovidorg/covid19testing-backend
# -----------------------

testing_df = pd.DataFrame(columns=['Name','Lat','Lon','City','State','Collecting','Evaluating'])
testing_df

base_url = "http://api.findcovidtesting.com/api/v1/location"
response = requests.get(base_url).json()

In [12]:
# -----------------------
# Use this to see how the response is structured:
#response[0]

# -----------------------
# Relevant info seems to be:
#     location_name
#     is_collecting_samples - boolean
#     is_evaluating_symptoms - boolean
#     location_address_locality = city
#     location_address_region = state
#     location_latitude
#     location_longitude


In [13]:
# Iterate through the response and store selected variables in dataframe

for i in range(len(response)):
    if response[i]['location_address_region'] == 'NC':
        testing_df.loc[i,'Name'] = response[i]['location_name']
        testing_df.loc[i,'Lat'] = response[i]['location_latitude']
        testing_df.loc[i,'Lon'] = response[i]['location_longitude']
        testing_df.loc[i,'City'] = response[i]['location_address_locality']
        testing_df.loc[i,'State'] = response[i]['location_address_region']
        testing_df.loc[i,'Collecting'] = response[i]['is_collecting_samples']
        testing_df.loc[i,'Evaluating'] = response[i]['is_evaluating_symptoms']

# Display the dataframe       
testing_df

,Name,Lat,Lon,City,State,Collecting,Evaluating
34,MEDAC Urgent Care - Military Cutoff,34.2249,-77.8305,Wilmington,NC,True,True
41,Roxboro MedPeds,36.4104,-78.9852,Roxboro,NC,True,True
51,Wayne County Health Department,35.3826,-77.9834,Goldsboro,NC,True,True
57,Range Urgent Care - Black Mountain,35.6154,-82.3185,,NC,True,True
59,FastMed Urgent Care,35.6523,-78.472,Clayton,NC,True,True
...,...,...,...,...,...,...,...
9898,Avance Care - Northeast Raleigh,35.8767,-78.5384,Raleigh,NC,True,True
9945,New Hanover County Public Health COVID-19 Coll...,34.2433,-77.9497,Wilmington,NC,True,True
9962,Avance Care - North Raleigh,35.9054,-78.7026,Raleigh,NC,True,True
9980,Duke Urgent Care - Harps Mill,35.8821,-78.6265,Raleigh,NC,True,True


In [14]:
gmaps.configure(api_key=google_api)

# Build the map
# Set center and zoom level
fig = gmaps.figure(map_type='HYBRID',center=(35.3,-79.5), zoom_level=7)
locations = testing_df[['Lat', 'Lon']]
hover = testing_df['Name'].tolist()

symbols = gmaps.symbol_layer(locations, fill_color='white', stroke_color='white', scale = 2, info_box_content = hover)
fig.add_layer(symbols)


# Display the map
fig

NameError: name 'google_api' is not defined

In [ ]:
# Upload and Combine NC County Demographic CSV's from https://www.census.gov/quickfacts/fact/table/US/PST045219

# Find all csv files using glob pattern matching with extension = 'csv' by saving results in a list titled all_county
os.chdir('./nc_county_demographics')
extension = 'csv'
all_county = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
nc_demographic = pd.concat([pd.read_csv(f) for f in all_county ])
nc_demographic = nc_demographic.dropna().rename(columns={'Fact': 'county'})

# Create New Dataframe to retrieve the demographic data per county
# Set Columns for new Dataframe
demo_summary = pd.DataFrame(columns=['County','% Female','% White','% Black','% Hispanic','% Population Under 18'
                                     '% Graduated Highschool','% College Graduate','People Per Household',
                                     'Median Household Income','Poverty Rate','% Without Health Insurance (<65)'])

# Set Values for New Demographic DataFrame
demo_summary['County'] = nc_demographic['county'].values
demo_summary['Female (%)'] = nc_demographic['Female persons, percent'].values
demo_summary['White (%)'] = nc_demographic['White alone, not Hispanic or Latino, percent'].values
demo_summary['Black (%)'] = nc_demographic['Black or African American alone, percent'].values
demo_summary['Hispanic (%)'] = nc_demographic['Hispanic or Latino, percent'].values
demo_summary['Population Under 18 (%)'] = nc_demographic['Persons under 18 years, percent'].values
demo_summary['Population 65 and Over (%)'] = nc_demographic['Persons 65 years and over, percent'].values
demo_summary['Graduated Highschool (%)'] = nc_demographic['High school graduate or higher, percent of persons age 25 years+, 2014-2018'].values
demo_summary['College Graduate (%)'] = nc_demographic['Bachelors degree or higher, percent of persons age 25 years+, 2014-2018'].values
demo_summary['Median Household Income'] = nc_demographic['Median household income (in 2018 dollars), 2014-2018'].values
demo_summary['Poverty Rate'] = nc_demographic['Persons in poverty, percent'].values
demo_summary['% Without Health Insurance (<65)'] = nc_demographic['Persons  without health insurance, under age 65 years, percent'].values

# Format New Demographic Data
demo_summary1 = demo_summary.transpose().dropna()
demo_summary2 = demo_summary1.transpose()
demo_summary2.sort_values('County', axis = 0, ascending = True, inplace = True)
demo_summary3 = demo_summary2.reset_index(drop = True)

# Add data from summary_df
demo_summary3['Population'] = summary_df['population'].values
demo_summary3['Cases'] = summary_df['cases'].values
demo_summary3['Deaths'] = summary_df['deaths'].values
demo_summary3['Cases/Population (%)'] = summary_df['cases_pop_pct'].values
demo_summary3['Cases Per 10,000 Residents'] = ((summary_df['cases'] / summary_df['population']) * 10000).values
demo_summary3['Deaths/Population (%)'] = summary_df['deaths_pop_pct'].values
demo_summary3['Deaths/Cases (%)'] = summary_df['deaths_cases_pct'].values


# Reorder Columns to look nice
demo_summary4 = demo_summary3[['County','Population','Female (%)','White (%)','Black (%)',
                               'Hispanic (%)','Population Under 18 (%)','Population 65 and Over (%)',
                               'Graduated Highschool (%)','College Graduate (%)','Median Household Income',
                               'Poverty Rate','% Without Health Insurance (<65)','Cases','Cases Per 10,000 Residents',
                               'Deaths','Cases/Population (%)','Deaths/Population (%)','Deaths/Cases (%)']]
demo_summary4.head(50)

In [ ]:
# Format Dataframe so column values work with regression equations
density_df = pd.read_csv('../07 20 2020 regression data.csv')


df = demo_summary4.astype(str)
df['Pop Density/mile'] = density_df['Pop Density/mile']

df.dtypes

for x in df:
    if x == "County":
        continue
    if(df[x].dtype == 'object'):
        if df[x].str.contains('%').any():
            df[x] = df[x].str.replace(',', '')
            df[x] = df[x].str.replace('%', '')
            df[x] = df[x].astype('float')
            continue
        df[x] = df[x].str.replace('$', '')
        df[x] = df[x].str.replace('.', '')
        df[x] = df[x].str.replace(',', '')
        df[x] = df[x].astype('float')
        
# Reset Cases Per 10,000 Residents Value
df['RF0'] = density_df['RF0']


df['Cases Per 10,000 Residents'] = ((summary_df['cases'] / summary_df['population']) * 10000).values

#density_df
df

In [ ]:
#Initial observations:
#As population increases, Cases per 10000 residents also seems to increase
#Whiter counties seem to have lower rates of covid
#Difficult to say at a glance whether or not education and median income have significant impacts
#Hypothesis: population should explain a reasonable amount of the variation in cases per 10,000 residents

In [ ]:
# Print out the r-squared value along with the plot.
x_values = df['Cases Per 10,000 Residents']
y_values = df['RF0']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")
plt.xlabel('Cases Per 10,000 Residents')
plt.ylabel('% of Population at Low Risk Factor')
plt.savefig(r'../output_data/fig1.png')
print(f"The r-squared is: {rvalue**2}")
plt.show()

In [ ]:
# Print out the r-squared value along with the plot.
x_values = df['Cases Per 10,000 Residents']
y_values = df['Population']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")
plt.xlabel('Cases Per 10,000 Residents')
plt.ylabel('Total Population')
plt.savefig(r'../output_data/fig1.png')
print(f"The r-squared is: {rvalue**2}")
plt.show()

In [ ]:
# Population seems to have very little effect on proportional cases, let's see if this changes if we drop 
# the most significant outlier, Duplin county.
df2 = df.drop(df.index[30])

x_values = df2['Cases Per 10,000 Residents']
y_values = df2['Population']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")
plt.xlabel('Cases Per 10,000 Residents')
plt.ylabel('Population')
plt.savefig(r'../output_data/fig2.png')
print(f"The r-squared is: {rvalue**2}")
plt.show()

In [ ]:
# Better, but population still explains less than 5% of the variance.
# Lets examinme the percentage of children in the population. There have been claims the the virus is spread most 
# prominently by children, however given the small variation in percentage of children in each county I would not 
# expect this to be a significant explanatory factor.

x_values = df['Cases Per 10,000 Residents']
y_values = df['Population Under 18 (%)']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.xlabel('Cases Per 10,000 Residents')
plt.ylabel('Population Under 18 (%)')
plt.savefig(r'../output_data/fig3.png')
print(f"The r-squared is: {rvalue**2}")
plt.show()

In [ ]:
# 17% is actually quite good considering the number of variables involved, and it does seem that children do indeed 
# spread the virus more. let's examine the percentage of senior citizens.

x_values = df['Cases Per 10,000 Residents']
y_values = df['Population 65 and Over (%)']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.xlabel('Cases Per 10,000 Residents')
plt.ylabel('Population 65 and Over (%)')
plt.savefig(r'../output_data/fig4.png')
print(f"The r-squared is: {rvalue**2}")
plt.show()

In [ ]:
# Significantly less of a factor than children, and higher percentages of elderly people have less instances of the 
# virus. Potentially a result of less children in these communities.
# Let's take a look at gender, which is unlikely to have a significant impact.

x_values = df['Cases Per 10,000 Residents']
y_values = df['Female (%)']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.xlabel('Cases Per 10,000 Residents')
plt.ylabel('Female (%)')
plt.savefig(r'../output_data/fig5.png')
print(f"The r-squared is: {rvalue**2}")
plt.show()

In [ ]:
# Unsurpisingly not much to see there.
# let's examine the percentage of white people in the county. Since the virus has disproportionately effected 
# minorities we would expect a negative correlation between the percentage of white people and the proportional cases.

x_values = df['Cases Per 10,000 Residents']
y_values = df['White (%)']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.xlabel('Cases Per 10,000 Residents')
plt.ylabel('White (%)')
plt.savefig(r'../output_data/fig6.png')
print(f"The r-squared is: {rvalue**2}")
plt.show()

In [ ]:
#Indeed there is a fairly significant negative correlation there.
#let's examine median income. Low income familes are likely to have less access to health care and live in more 
# crowded conditions, so we would imagine some negative correlation here.

x_values = df['Cases Per 10,000 Residents']
y_values = df['Median Household Income']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")
plt.xlabel('Cases Per 10,000 Residents')
plt.ylabel('Median Household income')
plt.savefig(r'../output_data/fig7.png')
print(f"The r-squared is: {rvalue**2}")
plt.show()

In [ ]:
# Almost no correlation at all. Let's take a look at poverty rate instead.

x_values = df['Cases Per 10,000 Residents']
y_values = df['Poverty Rate']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.xlabel('Cases Per 10,000 Residents')
plt.ylabel('Poverty Rate')
plt.savefig(r'../output_data/fig8.png')
print(f"The r-squared is: {rvalue**2}")
plt.show()

In [ ]:
#Better, but still a small correlation. Let's try education.
x_values = df['Cases Per 10,000 Residents']
y_values = df['Graduated Highschool (%)']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.xlabel('Cases Per 10,000 Residents')
plt.ylabel('Graduated Highschool (%)')
plt.savefig(r'../output_data/fig9.png')
print(f"The r-squared is: {rvalue**2}")
plt.show()

In [ ]:
x_values = df['Cases Per 10,000 Residents']
y_values = df['College Graduate (%)']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.xlabel('Cases Per 10,000 Residents')
plt.ylabel("College Graduate (%)")
plt.savefig(r'../output_data/fig10.png')              
print(f"The r-squared is: {rvalue**2}")
plt.show()

In [ ]:
df.head()
df.dtypes

In [ ]:
# A resonably high negative correlation for high school education and not much for college education.
# Now it's time to take a crack at multiple linear regression.
# No one demographic variable explains a particularly large amount of the variance in the proportional cases, 
# however together we should be able to explain a reasonable amount with a multiple linear regression.

x = df[['Pop Density/mile', 'Population Under 18 (%)', 'Population 65 and Over (%)', 'White (%)', 'Median Household Income', 'Graduated Highschool (%)', 'RF0']]

X = sm.add_constant(x) # adding a constant

model = sm.OLS(df["Cases Per 10,000 Residents"], X).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)

In [ ]:
x = df[['Pop Density/mile', 'Population Under 18 (%)', 'Population 65 and Over (%)', 'White (%)', 'Median Household Income', 'Graduated Highschool (%)', 'RF0']]

X = sm.add_constant(x) # adding a constant

model = sm.OLS(df["Deaths/Population (%)"], X).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)

In [ ]:
df2 = df[['Pop Density/mile', 'Population Under 18 (%)', 'Population 65 and Over (%)', 'White (%)', 'Median Household Income', 'Graduated Highschool (%)', 'RF0']]
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)
calc_vif(df2)